# Notebook 3: Test hybrid origin of Eriosyce chilensis

In [1]:
import ipyrad.analysis as ipa
import pandas as pd

### DATA FILES

In [2]:
# load RAD dataset
SNPS = "../assembly/Eriosyce-ref-2021_outfiles/Eriosyce-ref-2021.snps.hdf5"

# get names from the database
info = ipa.snps_extracter(SNPS)

### Sample metadata

In [3]:
df = pd.read_csv("../metadata/pablo-cacti-relabeled-re.csv")
df.head(10)

,accession,Genus,specie,subspecie,Localidad,Lat,Long,oldname
0,1075,Eriosyce,litoralis,NaN,TotoralilloIVR,-30.068,-71.375,litoralis
1,1083,Eriosyce,litoralis,NaN,TotoralilloIVR,-30.069,-71.376,litoralis
2,1093,Eriosyce,clavata,NaN,Qda.Manqueza,-29.914,-70.943,clavata
3,1288,Eriosyce,subgibbosa,NaN,PeninsuladeHualpenVIIIR,-36.757,-73.176,subgibbosa
4,1290,Eriosyce,subgibbosa,NaN,PeninsuladeHualpenVIIIR,-36.757,-73.176,subgibbosa
5,1332,Eriosyce,castanea,NaN,CerroLaLajuela(StaCruz),-34.664,-71.417,castanea
6,1333,Eriosyce,castanea,NaN,CerroLaLajuela(StaCruz),-34.664,-71.417,castanea
7,1349,Eriosyce,litoralis,NaN,Pichidanguii,-32.156,-71.528,subgibbosa
8,1350,Eriosyce,litoralis,NaN,Pichidanguii,-32.156,-71.528,subgibbosa
9,1351,Eriosyce,litoralis,NaN,Pichidanguii,-32.156,-71.528,subgibbosa


## Setup ABBA-BABA tool

In [4]:
tool = ipa.baba2(data="../assembly/Eriosyce-ref-2021_outfiles/Eriosyce-ref-2021.snps.hdf5")

In [5]:
tool.ipcluster['cores'] = 25

### Full sampling
BV190 is the outgroup (marksiana)

In [6]:
lito_allo = df[(df.specie == "litoralis") & (df.oldname != "subgibbosa")].accession.tolist()
lito = df[(df.specie == "litoralis") & (df.oldname == "subgibbosa")].accession.tolist()
muta = df[(df.specie == "mutabilis")].accession.tolist()
chil = df[(df.specie == "chilensis")].accession.tolist()
albi = df[(df.specie == "chilensis-albidiflora")].accession.tolist()

In [7]:
IMAPS = [
    {
        "p4": ["BV190"],
        "p3": lito,                                        # lito
        "p2": ["HPG1", "HPG2", "HPG4"],                    # lit-x-mutab
        "p1": muta,                                        # mutab
    },
    {
        "p4": ["BV190"],
        "p3": lito,                                        # lito
        "p2": ["SXM1", "SXM2"],                            # lit-x-mutab
        "p1": muta,                                        # mutab
    },
    {
        "p4": ["BV190"],
        "p3": muta,            # mutab
        "p2": lito,            # lito
        "p1": chil,            # chil
    },
    {
        "p4": ["BV190"],
        "p3": muta,            # mutab
        "p2": lito_allo,            # lito
        "p1": chil,            # chil
    },
    {
        "p4": ["BV190"],
        "p3": muta,           # mutab
        "p2": lito,            # lito
        "p1": albi,            # chil-alb
    },
    {
        "p4": ["BV190"],
        "p3": muta,           # mutab
        "p2": lito_allo,            # lito
        "p1": albi,            # chil-alb
    },
    {
        "p4": ["BV190"],
        "p3": muta,            # mut
        "p2": chil,            # chil
        "p1": albi,            # chil-alb
    },
    {
        "p4": ["BV190"],
        "p3": lito,            # lito
        "p2": chil,            # chil
        "p1": albi,            # chil-alb
    },
    {
        "p4": ["BV190"],
        "p3": lito_allo,       # lito
        "p2": chil,            # chil
        "p1": albi,            # chil-alb
    },
    {
        "p1": lito_allo,
        "p2": lito,
        "p3": muta,
        "p4": ["BV190"], 
    }
]

#### Requre at >=50% of samples in each clade to have data 

In [8]:
tool.run(
    imaps=IMAPS,
    minmaps=[{i: 0.5 for i in ('p1', 'p2', 'p3', 'p4')} for i in IMAPS],
    nboots=1000,
)

[####################] 100% 0:59:40 | abba-baba tests 


In [10]:
tool.results_table

,D,bootstd,Z,ABBA,BABA,nSNPs,nloci
0,0.340,0.023,15.013,431.695,212.855,37606,4117
1,0.521,0.022,23.517,972.273,306.534,52921,5923
2,-0.094,0.013,7.163,271.960,328.302,56527,5483
3,-0.236,0.023,10.404,255.703,413.290,45804,5393
4,-0.098,0.011,9.158,341.929,415.825,69307,6868
5,-0.221,0.023,9.549,338.486,530.334,55413,6699
6,-0.021,0.011,1.862,284.161,296.283,51813,5434
7,-0.028,0.010,2.804,433.798,458.686,54785,5955
8,-0.009,0.012,0.721,355.136,361.478,43655,5802
9,0.143,0.023,6.120,478.019,358.422,65285,7210
